In [1]:
#Import required libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score,classification_report
import pickle

In [2]:
#Import datasets
data1=pd.read_csv(r"C:\Users\hp\Downloads\IPL project\IPL project\IPL_Ball_by_Ball_2008_2022.csv")
ball=data1.copy()
ball.head()

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals


In [3]:
data2=pd.read_csv(r"C:\Users\hp\Downloads\IPL project\IPL project\IPL_Matches_2008_2022.csv")
matches=data2.copy()
matches.head()

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,SuperOver,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2
0,1312200,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon
1,1312199,Ahmedabad,2022-05-27,2022,Qualifier 2,Royal Challengers Bangalore,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,field,N,Rajasthan Royals,Wickets,7.0,NaN,JC Buttler,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,Nitin Menon
2,1312198,Kolkata,2022-05-25,2022,Eliminator,Royal Challengers Bangalore,Lucknow Super Giants,"Eden Gardens, Kolkata",Lucknow Super Giants,field,N,Royal Challengers Bangalore,Runs,14.0,NaN,RM Patidar,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['Q de Kock', 'KL Rahul', 'M Vohra', 'DJ Hooda...",J Madanagopal,MA Gough
3,1312197,Kolkata,2022-05-24,2022,Qualifier 1,Rajasthan Royals,Gujarat Titans,"Eden Gardens, Kolkata",Gujarat Titans,field,N,Gujarat Titans,Wickets,7.0,NaN,DA Miller,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",BNJ Oxenford,VK Sharma
4,1304116,Mumbai,2022-05-22,2022,70,Sunrisers Hyderabad,Punjab Kings,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,bat,N,Punjab Kings,Wickets,5.0,NaN,Harpreet Brar,"['PK Garg', 'Abhishek Sharma', 'RA Tripathi', ...","['JM Bairstow', 'S Dhawan', 'M Shahrukh Khan',...",AK Chaudhary,NA Patwardhan


In [4]:
#Data Cleaning / Data Manipulation

In [5]:
#Manipulating "Season" by replacing years
matches["Season"]=matches["Season"].replace(["2007/08","2009/10","2020/21"],["2008","2010","2020"])

In [6]:
#Find unique teams in dataset
team=pd.concat([matches['Team1'], matches['Team2']])
teams=set(team)

In [7]:
teams

{'Chennai Super Kings',
 'Deccan Chargers',
 'Delhi Capitals',
 'Delhi Daredevils',
 'Gujarat Lions',
 'Gujarat Titans',
 'Kings XI Punjab',
 'Kochi Tuskers Kerala',
 'Kolkata Knight Riders',
 'Lucknow Super Giants',
 'Mumbai Indians',
 'Pune Warriors',
 'Punjab Kings',
 'Rajasthan Royals',
 'Rising Pune Supergiant',
 'Rising Pune Supergiants',
 'Royal Challengers Bangalore',
 'Sunrisers Hyderabad'}

In [8]:
#Since some teams have changed their team names , replaced with only one name
teams_rename={"Delhi Daredevils":"Delhi Capitals",'Rising Pune Supergiants':'Rising Pune Supergiant','Kings XI Punjab':"Punjab Kings","Pune Warriors":"Rising Pune Supergiant"}

In [9]:
matches.replace({"Team1": teams_rename, "Team2": teams_rename,"WinningTeam":teams_rename}, inplace=True)


In [10]:
team1=pd.concat([matches['Team1'], matches['Team2']])
new_teams=set(team1)

In [11]:
new_teams

{'Chennai Super Kings',
 'Deccan Chargers',
 'Delhi Capitals',
 'Gujarat Lions',
 'Gujarat Titans',
 'Kochi Tuskers Kerala',
 'Kolkata Knight Riders',
 'Lucknow Super Giants',
 'Mumbai Indians',
 'Punjab Kings',
 'Rajasthan Royals',
 'Rising Pune Supergiant',
 'Royal Challengers Bangalore',
 'Sunrisers Hyderabad'}

In [12]:
#Renaming teams in "ball" dataset.
ball.replace({"BattingTeam": teams_rename}, inplace=True)


In [13]:
#Calculate total_runs of each team in each match based on ID
total_runs = ball.groupby(['ID','innings']).sum()['total_run'].reset_index()

In [14]:
total_runs

,ID,innings,total_run
0,335982,1,222
1,335982,2,82
2,335983,1,240
3,335983,2,207
4,335984,1,129
...,...,...,...
1923,1312198,2,193
1924,1312199,1,157
1925,1312199,2,161
1926,1312200,1,130


In [15]:
#Calculate target for teams that has to be played in "Innings"==2
for ID in total_runs['ID'].unique():
    # Filter DataFrame for innings 1
    inning_1 = total_runs[(total_runs['ID'] == ID) & (total_runs['innings'] == 1)]
    # Calculate target for innings 2
    target_for_innings_2 = inning_1['total_run'].iloc[0] + 1
    # Update target for innings 2 in the DataFrame
    total_runs.loc[(total_runs['ID'] == ID) & (total_runs['innings'] == 2), 'target'] = target_for_innings_2

print(total_runs)

           ID  innings  total_run  target
0      335982        1        222     NaN
1      335982        2         82   223.0
2      335983        1        240     NaN
3      335983        2        207   241.0
4      335984        1        129     NaN
...       ...      ...        ...     ...
1923  1312198        2        193   208.0
1924  1312199        1        157     NaN
1925  1312199        2        161   158.0
1926  1312200        1        130     NaN
1927  1312200        2        133   131.0

[1928 rows x 4 columns]


In [16]:
#Since the Innings==1 doesn't have any target , filled with "0"
total_runs["target"]=total_runs["target"].fillna(0)

In [17]:
total_runs

,ID,innings,total_run,target
0,335982,1,222,0.0
1,335982,2,82,223.0
2,335983,1,240,0.0
3,335983,2,207,241.0
4,335984,1,129,0.0
...,...,...,...,...
1923,1312198,2,193,208.0
1924,1312199,1,157,0.0
1925,1312199,2,161,158.0
1926,1312200,1,130,0.0


In [18]:
total_runs = total_runs[total_runs['innings']==2]

In [19]:
total_runs

,ID,innings,total_run,target
1,335982,2,82,223.0
3,335983,2,207,241.0
5,335984,2,132,130.0
7,335985,2,166,166.0
9,335986,2,112,111.0
...,...,...,...,...
1919,1304116,2,160,158.0
1921,1312197,2,191,189.0
1923,1312198,2,193,208.0
1925,1312199,2,161,158.0


In [20]:
#Merging datasets "matches" and "total_runs" on "ID"

In [21]:
match_df = matches.merge(total_runs[['ID','total_run','target']],on="ID")

In [22]:
match_df

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,...,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2,total_run,target
0,1312200,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,...,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,133,131.0
1,1312199,Ahmedabad,2022-05-27,2022,Qualifier 2,Royal Challengers Bangalore,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,field,...,Wickets,7.0,NaN,JC Buttler,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,Nitin Menon,161,158.0
2,1312198,Kolkata,2022-05-25,2022,Eliminator,Royal Challengers Bangalore,Lucknow Super Giants,"Eden Gardens, Kolkata",Lucknow Super Giants,field,...,Runs,14.0,NaN,RM Patidar,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['Q de Kock', 'KL Rahul', 'M Vohra', 'DJ Hooda...",J Madanagopal,MA Gough,193,208.0
3,1312197,Kolkata,2022-05-24,2022,Qualifier 1,Rajasthan Royals,Gujarat Titans,"Eden Gardens, Kolkata",Gujarat Titans,field,...,Wickets,7.0,NaN,DA Miller,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",BNJ Oxenford,VK Sharma,191,189.0
4,1304116,Mumbai,2022-05-22,2022,70,Sunrisers Hyderabad,Punjab Kings,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,bat,...,Wickets,5.0,NaN,Harpreet Brar,"['PK Garg', 'Abhishek Sharma', 'RA Tripathi', ...","['JM Bairstow', 'S Dhawan', 'M Shahrukh Khan',...",AK Chaudhary,NA Patwardhan,160,158.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,335986,Kolkata,2008-04-20,2008,4,Kolkata Knight Riders,Deccan Chargers,Eden Gardens,Deccan Chargers,bat,...,Wickets,5.0,NaN,DJ Hussey,"['WP Saha', 'BB McCullum', 'RT Ponting', 'SC G...","['AC Gilchrist', 'Y Venugopal Rao', 'VVS Laxma...",BF Bowden,K Hariharan,112,111.0
944,335985,Mumbai,2008-04-20,2008,5,Mumbai Indians,Royal Challengers Bangalore,Wankhede Stadium,Mumbai Indians,bat,...,Wickets,5.0,NaN,MV Boucher,"['L Ronchi', 'ST Jayasuriya', 'DJ Thornely', '...","['S Chanderpaul', 'R Dravid', 'LRPL Taylor', '...",SJ Davis,DJ Harper,166,166.0
945,335984,Delhi,2008-04-19,2008,3,Delhi Capitals,Rajasthan Royals,Feroz Shah Kotla,Rajasthan Royals,bat,...,Wickets,9.0,NaN,MF Maharoof,"['G Gambhir', 'V Sehwag', 'S Dhawan', 'MK Tiwa...","['T Kohli', 'YK Pathan', 'SR Watson', 'M Kaif'...",Aleem Dar,GA Pratapkumar,132,130.0
946,335983,Chandigarh,2008-04-19,2008,2,Punjab Kings,Chennai Super Kings,"Punjab Cricket Association Stadium, Mohali",Chennai Super Kings,bat,...,Runs,33.0,NaN,MEK Hussey,"['K Goel', 'JR Hopes', 'KC Sangakkara', 'Yuvra...","['PA Patel', 'ML Hayden', 'MEK Hussey', 'MS Dh...",MR Benson,SL Shastri,207,241.0


In [23]:
# Since some teams were not played in maximum seasons ,considering only some teams for further process

In [24]:
teams = ['Kolkata Knight Riders', 
         'Chennai Super Kings',
         'Punjab Kings', 
         'Rajasthan Royals',
         'Gujarat Titans',
         'Mumbai Indians',
         'Delhi Capitals',
         'Royal Challengers Bangalore',
         'Sunrisers Hyderabad']

In [25]:
# Datasets with required teams
match_df = match_df[match_df['Team1'].isin(teams)]
match_df = match_df[match_df['Team2'].isin(teams)]
ball=ball[ball["BattingTeam"].isin(teams)]

In [26]:
teams=pd.concat([match_df['Team1'], match_df['Team2']])
n_teams=set(teams)
n_teams

{'Chennai Super Kings',
 'Delhi Capitals',
 'Gujarat Titans',
 'Kolkata Knight Riders',
 'Mumbai Indians',
 'Punjab Kings',
 'Rajasthan Royals',
 'Royal Challengers Bangalore',
 'Sunrisers Hyderabad'}

In [27]:
match_df.shape

(749, 22)

In [28]:
match_df.head(10)

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,...,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2,total_run,target
0,1312200,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,...,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,133,131.0
1,1312199,Ahmedabad,2022-05-27,2022,Qualifier 2,Royal Challengers Bangalore,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,field,...,Wickets,7.0,NaN,JC Buttler,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,Nitin Menon,161,158.0
3,1312197,Kolkata,2022-05-24,2022,Qualifier 1,Rajasthan Royals,Gujarat Titans,"Eden Gardens, Kolkata",Gujarat Titans,field,...,Wickets,7.0,NaN,DA Miller,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",BNJ Oxenford,VK Sharma,191,189.0
4,1304116,Mumbai,2022-05-22,2022,70,Sunrisers Hyderabad,Punjab Kings,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,bat,...,Wickets,5.0,NaN,Harpreet Brar,"['PK Garg', 'Abhishek Sharma', 'RA Tripathi', ...","['JM Bairstow', 'S Dhawan', 'M Shahrukh Khan',...",AK Chaudhary,NA Patwardhan,160,158.0
5,1304115,Mumbai,2022-05-21,2022,69,Delhi Capitals,Mumbai Indians,"Wankhede Stadium, Mumbai",Mumbai Indians,field,...,Wickets,5.0,NaN,JJ Bumrah,"['PP Shaw', 'DA Warner', 'MR Marsh', 'RR Pant'...","['Ishan Kishan', 'RG Sharma', 'D Brevis', 'Til...",Nitin Menon,Tapan Sharma,160,160.0
6,1304114,Mumbai,2022-05-20,2022,68,Chennai Super Kings,Rajasthan Royals,"Brabourne Stadium, Mumbai",Chennai Super Kings,bat,...,Wickets,5.0,NaN,R Ashwin,"['RD Gaikwad', 'DP Conway', 'MM Ali', 'N Jagad...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,NA Patwardhan,151,151.0
7,1304113,Mumbai,2022-05-19,2022,67,Gujarat Titans,Royal Challengers Bangalore,"Wankhede Stadium, Mumbai",Gujarat Titans,bat,...,Wickets,8.0,NaN,V Kohli,"['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...","['V Kohli', 'F du Plessis', 'GJ Maxwell', 'KD ...",KN Ananthapadmanabhan,GR Sadashiv Iyer,170,169.0
9,1304111,Mumbai,2022-05-17,2022,65,Sunrisers Hyderabad,Mumbai Indians,"Wankhede Stadium, Mumbai",Mumbai Indians,field,...,Runs,3.0,NaN,RA Tripathi,"['Abhishek Sharma', 'PK Garg', 'RA Tripathi', ...","['RG Sharma', 'Ishan Kishan', 'DR Sams', 'Tila...",CB Gaffaney,N Pandit,190,194.0
10,1304110,Navi Mumbai,2022-05-16,2022,64,Delhi Capitals,Punjab Kings,"Dr DY Patil Sports Academy, Mumbai",Punjab Kings,field,...,Runs,17.0,NaN,SN Thakur,"['DA Warner', 'SN Khan', 'MR Marsh', 'Lalit Ya...","['JM Bairstow', 'S Dhawan', 'PBB Rajapaksa', '...",GR Sadashiv Iyer,Nitin Menon,142,160.0
12,1304108,Mumbai,2022-05-15,2022,62,Chennai Super Kings,Gujarat Titans,"Wankhede Stadium, Mumbai",Chennai Super Kings,bat,...,Wickets,7.0,NaN,WP Saha,"['RD Gaikwad', 'DP Conway', 'MM Ali', 'N Jagad...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",R Pandit,VK Sharma,137,134.0


In [29]:
match_df.columns

Index(['ID', 'City', 'Date', 'Season', 'MatchNumber', 'Team1', 'Team2',
       'Venue', 'TossWinner', 'TossDecision', 'SuperOver', 'WinningTeam',
       'WonBy', 'Margin', 'method', 'Player_of_Match', 'Team1Players',
       'Team2Players', 'Umpire1', 'Umpire2', 'total_run', 'target'],
      dtype='object')

In [30]:
#Getting required columns from "match_df" 
match_df = match_df[['ID','City','TossDecision','Team1','Team2','WinningTeam','total_run','target',]]

In [31]:
#Merging "match_df" and "ball" on "ID"

In [32]:

match_df2= match_df.merge(ball,on='ID')
match_df2

,ID,City,TossDecision,Team1,Team2,WinningTeam,total_run_x,target,innings,overs,...,extra_type,batsman_run,extras_run,total_run_y,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
0,1312200,Ahmedabad,bat,Rajasthan Royals,Gujarat Titans,Gujarat Titans,133,131.0,1,0,...,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
1,1312200,Ahmedabad,bat,Rajasthan Royals,Gujarat Titans,Gujarat Titans,133,131.0,1,0,...,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals
2,1312200,Ahmedabad,bat,Rajasthan Royals,Gujarat Titans,Gujarat Titans,133,131.0,1,0,...,NaN,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals
3,1312200,Ahmedabad,bat,Rajasthan Royals,Gujarat Titans,Gujarat Titans,133,131.0,1,0,...,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
4,1312200,Ahmedabad,bat,Rajasthan Royals,Gujarat Titans,Gujarat Titans,133,131.0,1,0,...,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178245,335982,Bangalore,field,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,82,223.0,2,14,...,legbyes,0,1,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore
178246,335982,Bangalore,field,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,82,223.0,2,14,...,NaN,1,0,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore
178247,335982,Bangalore,field,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,82,223.0,2,14,...,NaN,0,0,0,0,0,NaN,NaN,NaN,Royal Challengers Bangalore
178248,335982,Bangalore,field,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,82,223.0,2,15,...,wides,0,1,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore


In [33]:
#Getting dataframe with where "innings"==2
match_df2= match_df2[match_df2['innings']==2]

In [34]:
match_df2.head()

,ID,City,TossDecision,Team1,Team2,WinningTeam,total_run_x,target,innings,overs,...,extra_type,batsman_run,extras_run,total_run_y,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
120,1312200,Ahmedabad,bat,Rajasthan Royals,Gujarat Titans,Gujarat Titans,133,131.0,2,0,...,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans
121,1312200,Ahmedabad,bat,Rajasthan Royals,Gujarat Titans,Gujarat Titans,133,131.0,2,0,...,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans
122,1312200,Ahmedabad,bat,Rajasthan Royals,Gujarat Titans,Gujarat Titans,133,131.0,2,0,...,NaN,1,0,1,0,0,NaN,NaN,NaN,Gujarat Titans
123,1312200,Ahmedabad,bat,Rajasthan Royals,Gujarat Titans,Gujarat Titans,133,131.0,2,0,...,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans
124,1312200,Ahmedabad,bat,Rajasthan Royals,Gujarat Titans,Gujarat Titans,133,131.0,2,0,...,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans


In [35]:
ball.columns

Index(['ID', 'innings', 'overs', 'ballnumber', 'batter', 'bowler',
       'non-striker', 'extra_type', 'batsman_run', 'extras_run', 'total_run',
       'non_boundary', 'isWicketDelivery', 'player_out', 'kind',
       'fielders_involved', 'BattingTeam'],
      dtype='object')

In [36]:
#Define a function to add bowling team
def bowling_team(team):
    if team["BattingTeam"]==team["Team1"]:
        return team["Team2"]
    else:
        return team["Team1"]


In [37]:
#Adding column "bowling team" to "match_df2"
match_df2["BowlingTeam"]=match_df2.apply(bowling_team,axis=1)

In [38]:
match_df2

,ID,City,TossDecision,Team1,Team2,WinningTeam,total_run_x,target,innings,overs,...,batsman_run,extras_run,total_run_y,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,BowlingTeam
120,1312200,Ahmedabad,bat,Rajasthan Royals,Gujarat Titans,Gujarat Titans,133,131.0,2,0,...,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,Rajasthan Royals
121,1312200,Ahmedabad,bat,Rajasthan Royals,Gujarat Titans,Gujarat Titans,133,131.0,2,0,...,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,Rajasthan Royals
122,1312200,Ahmedabad,bat,Rajasthan Royals,Gujarat Titans,Gujarat Titans,133,131.0,2,0,...,1,0,1,0,0,NaN,NaN,NaN,Gujarat Titans,Rajasthan Royals
123,1312200,Ahmedabad,bat,Rajasthan Royals,Gujarat Titans,Gujarat Titans,133,131.0,2,0,...,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,Rajasthan Royals
124,1312200,Ahmedabad,bat,Rajasthan Royals,Gujarat Titans,Gujarat Titans,133,131.0,2,0,...,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,Rajasthan Royals
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178245,335982,Bangalore,field,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,82,223.0,2,14,...,0,1,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore,Kolkata Knight Riders
178246,335982,Bangalore,field,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,82,223.0,2,14,...,1,0,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore,Kolkata Knight Riders
178247,335982,Bangalore,field,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,82,223.0,2,14,...,0,0,0,0,0,NaN,NaN,NaN,Royal Challengers Bangalore,Kolkata Knight Riders
178248,335982,Bangalore,field,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,82,223.0,2,15,...,0,1,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore,Kolkata Knight Riders


In [39]:
match_df2['total_run_y'] = pd.to_numeric(match_df2['total_run_y'], errors='raise')


In [40]:
#Adding column "current_score" using groupby 
match_df2['current_Score'] = match_df2.groupby('ID')['total_run_y'].cumsum()



In [41]:
match_df2.T

,120,121,122,123,124,125,126,127,128,129,...,178240,178241,178242,178243,178244,178245,178246,178247,178248,178249
ID,1312200,1312200,1312200,1312200,1312200,1312200,1312200,1312200,1312200,1312200,...,335982,335982,335982,335982,335982,335982,335982,335982,335982,335982
City,Ahmedabad,Ahmedabad,Ahmedabad,Ahmedabad,Ahmedabad,Ahmedabad,Ahmedabad,Ahmedabad,Ahmedabad,Ahmedabad,...,Bangalore,Bangalore,Bangalore,Bangalore,Bangalore,Bangalore,Bangalore,Bangalore,Bangalore,Bangalore
TossDecision,bat,bat,bat,bat,bat,bat,bat,bat,bat,bat,...,field,field,field,field,field,field,field,field,field,field
Team1,Rajasthan Royals,Rajasthan Royals,Rajasthan Royals,Rajasthan Royals,Rajasthan Royals,Rajasthan Royals,Rajasthan Royals,Rajasthan Royals,Rajasthan Royals,Rajasthan Royals,...,Royal Challengers Bangalore,Royal Challengers Bangalore,Royal Challengers Bangalore,Royal Challengers Bangalore,Royal Challengers Bangalore,Royal Challengers Bangalore,Royal Challengers Bangalore,Royal Challengers Bangalore,Royal Challengers Bangalore,Royal Challengers Bangalore
Team2,Gujarat Titans,Gujarat Titans,Gujarat Titans,Gujarat Titans,Gujarat Titans,Gujarat Titans,Gujarat Titans,Gujarat Titans,Gujarat Titans,Gujarat Titans,...,Kolkata Knight Riders,Kolkata Knight Riders,Kolkata Knight Riders,Kolkata Knight Riders,Kolkata Knight Riders,Kolkata Knight Riders,Kolkata Knight Riders,Kolkata Knight Riders,Kolkata Knight Riders,Kolkata Knight Riders
WinningTeam,Gujarat Titans,Gujarat Titans,Gujarat Titans,Gujarat Titans,Gujarat Titans,Gujarat Titans,Gujarat Titans,Gujarat Titans,Gujarat Titans,Gujarat Titans,...,Kolkata Knight Riders,Kolkata Knight Riders,Kolkata Knight Riders,Kolkata Knight Riders,Kolkata Knight Riders,Kolkata Knight Riders,Kolkata Knight Riders,Kolkata Knight Riders,Kolkata Knight Riders,Kolkata Knight Riders
total_run_x,133,133,133,133,133,133,133,133,133,133,...,82,82,82,82,82,82,82,82,82,82
target,131.0,131.0,131.0,131.0,131.0,131.0,131.0,131.0,131.0,131.0,...,223.0,223.0,223.0,223.0,223.0,223.0,223.0,223.0,223.0,223.0
innings,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
overs,0,0,0,0,0,0,1,1,1,1,...,13,14,14,14,14,14,14,14,15,15


In [42]:
#Calculating "runs_left" to win the match
match_df2['runs_left'] =np.where( (match_df2['target'] - match_df2['current_Score'])>=0,match_df2['target'] - match_df2['current_Score'],0)


In [43]:
#Calculating "balls_left" from overall balls
match_df2['balls_left'] =np.where(120 - ((match_df2['overs'])*6 +match_df2['ballnumber'])>=0,120 - ((match_df2['overs'])*6 +match_df2['ballnumber']),0)

## Getting wickets

In [44]:
#Data manipulation to get wickets 
match_df2['player_out'] = match_df2['player_out'].fillna("0")

In [45]:
match_df2['player_out'] = match_df2['player_out'].apply(lambda x:x if x=="0" else "1")

In [46]:
match_df2['player_out'] = match_df2['player_out'].astype('int')

In [47]:
wickets=match_df2['wickets'] = match_df2.groupby('ID')['player_out'].cumsum()


In [48]:
wickets

120        0
121        0
122        0
123        0
124        0
          ..
178245     9
178246     9
178247     9
178248     9
178249    10
Name: player_out, Length: 85939, dtype: int32

In [49]:
#Wicets _remaining to the team 
match_df2['wickets_remaining'] = 10-wickets

In [50]:
match_df2.head()

,ID,City,TossDecision,Team1,Team2,WinningTeam,total_run_x,target,innings,overs,...,player_out,kind,fielders_involved,BattingTeam,BowlingTeam,current_Score,runs_left,balls_left,wickets,wickets_remaining
120,1312200,Ahmedabad,bat,Rajasthan Royals,Gujarat Titans,Gujarat Titans,133,131.0,2,0,...,0,NaN,NaN,Gujarat Titans,Rajasthan Royals,0,131.0,119,0,10
121,1312200,Ahmedabad,bat,Rajasthan Royals,Gujarat Titans,Gujarat Titans,133,131.0,2,0,...,0,NaN,NaN,Gujarat Titans,Rajasthan Royals,0,131.0,118,0,10
122,1312200,Ahmedabad,bat,Rajasthan Royals,Gujarat Titans,Gujarat Titans,133,131.0,2,0,...,0,NaN,NaN,Gujarat Titans,Rajasthan Royals,1,130.0,117,0,10
123,1312200,Ahmedabad,bat,Rajasthan Royals,Gujarat Titans,Gujarat Titans,133,131.0,2,0,...,0,NaN,NaN,Gujarat Titans,Rajasthan Royals,1,130.0,116,0,10
124,1312200,Ahmedabad,bat,Rajasthan Royals,Gujarat Titans,Gujarat Titans,133,131.0,2,0,...,0,NaN,NaN,Gujarat Titans,Rajasthan Royals,1,130.0,115,0,10


In [51]:
#Calculating current run rate
match_df2['crr'] = round((match_df2['current_Score']*6)/(120 - match_df2['balls_left']),2)

In [52]:
#Calculating required run rate
match_df2['rrr'] = np.where(match_df2["balls_left"]>0,round((match_df2['runs_left']*6)/(match_df2['balls_left']),2),0)

In [53]:
#Define a function to get result of the play 
def winner(team):
    return 1 if team['BattingTeam'] == team['WinningTeam'] else 0

In [54]:
match_df2['results'] = match_df2.apply(winner,axis=1)

In [55]:
match_df2

,ID,City,TossDecision,Team1,Team2,WinningTeam,total_run_x,target,innings,overs,...,BattingTeam,BowlingTeam,current_Score,runs_left,balls_left,wickets,wickets_remaining,crr,rrr,results
120,1312200,Ahmedabad,bat,Rajasthan Royals,Gujarat Titans,Gujarat Titans,133,131.0,2,0,...,Gujarat Titans,Rajasthan Royals,0,131.0,119,0,10,0.00,6.61,1
121,1312200,Ahmedabad,bat,Rajasthan Royals,Gujarat Titans,Gujarat Titans,133,131.0,2,0,...,Gujarat Titans,Rajasthan Royals,0,131.0,118,0,10,0.00,6.66,1
122,1312200,Ahmedabad,bat,Rajasthan Royals,Gujarat Titans,Gujarat Titans,133,131.0,2,0,...,Gujarat Titans,Rajasthan Royals,1,130.0,117,0,10,2.00,6.67,1
123,1312200,Ahmedabad,bat,Rajasthan Royals,Gujarat Titans,Gujarat Titans,133,131.0,2,0,...,Gujarat Titans,Rajasthan Royals,1,130.0,116,0,10,1.50,6.72,1
124,1312200,Ahmedabad,bat,Rajasthan Royals,Gujarat Titans,Gujarat Titans,133,131.0,2,0,...,Gujarat Titans,Rajasthan Royals,1,130.0,115,0,10,1.20,6.78,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178245,335982,Bangalore,field,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,82,223.0,2,14,...,Royal Challengers Bangalore,Kolkata Knight Riders,80,143.0,31,9,1,5.39,27.68,0
178246,335982,Bangalore,field,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,82,223.0,2,14,...,Royal Challengers Bangalore,Kolkata Knight Riders,81,142.0,30,9,1,5.40,28.40,0
178247,335982,Bangalore,field,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,82,223.0,2,14,...,Royal Challengers Bangalore,Kolkata Knight Riders,81,142.0,29,9,1,5.34,29.38,0
178248,335982,Bangalore,field,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,82,223.0,2,15,...,Royal Challengers Bangalore,Kolkata Knight Riders,82,141.0,29,9,1,5.41,29.17,0


In [56]:
#Taking required columns that helps for prediction 
final_df =match_df2[['BattingTeam','BowlingTeam','runs_left','balls_left','wickets_remaining','target','crr','rrr','results']]

In [57]:
final_df.shape

(85939, 9)

In [58]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85939 entries, 120 to 178249
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   BattingTeam        85939 non-null  object 
 1   BowlingTeam        85939 non-null  object 
 2   runs_left          85939 non-null  float64
 3   balls_left         85939 non-null  int64  
 4   wickets_remaining  85939 non-null  int32  
 5   target             85939 non-null  float64
 6   crr                85939 non-null  float64
 7   rrr                85939 non-null  float64
 8   results            85939 non-null  int64  
dtypes: float64(4), int32(1), int64(2), object(2)
memory usage: 8.2+ MB


In [59]:
final_df.isnull().sum()

BattingTeam          0
BowlingTeam          0
runs_left            0
balls_left           0
wickets_remaining    0
target               0
crr                  0
rrr                  0
results              0
dtype: int64

In [60]:
final_df["results"].value_counts()

results
1    45999
0    39940
Name: count, dtype: int64

In [61]:
#Saving the data
final_df.to_csv(r"C:\Users\hp\Downloads\IPL project\final_ipl_data1.csv", index=False)

In [62]:
#Feature engineering

In [63]:
#Extract feature variables and class variable
fv= final_df.iloc[:, :-1]
cv= final_df.iloc[:,-1]

In [64]:
# Splitting tha data to train and test splits
x_train,x_test,y_train,y_test = train_test_split(fv,cv,test_size=0.20,random_state=3,stratify=cv)


In [65]:
#Dividing te dataset based on the type if the variables
numerical_data=x_train.select_dtypes(include=["int64","float64"])
cat_data=x_train.select_dtypes(include=["object"])

In [66]:
#Pipeline to impute and scale numerical columns
num_p=Pipeline([("imputing_n",SimpleImputer()),("scaling",StandardScaler())])

In [67]:
#Pipeline to impute and Encode nominal columns
cp=Pipeline([("imputing_c",SimpleImputer(strategy="most_frequent")),("Encoder",OneHotEncoder())])

In [68]:
#Pipeline for column transformation to apply for different types of data
ct=ColumnTransformer([("nominal",cp,cat_data.columns),("numerical",num_p,numerical_data.columns)],remainder="passthrough")

In [69]:
ct

ColumnTransformer(remainder='passthrough',
                  transformers=[('nominal',
                                 Pipeline(steps=[('imputing_c',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('Encoder', OneHotEncoder())]),
                                 Index(['BattingTeam', 'BowlingTeam'], dtype='object')),
                                ('numerical',
                                 Pipeline(steps=[('imputing_n',
                                                  SimpleImputer()),
                                                 ('scaling',
                                                  StandardScaler())]),
                                 Index(['runs_left', 'balls_left', 'target', 'crr', 'rrr'], dtype='object'))])

In [70]:


# Define pipelines
pipelines = {

    'logistic_regression': Pipeline([('classifier', LogisticRegression())]),
    'random_forest': Pipeline([('classifier', RandomForestClassifier())]),
    'decision_tree': Pipeline([('classifier', DecisionTreeClassifier())]),
}

# Define parameter grid for each algorithm
param_grids = {


    'logistic_regression': [
        {'classifier__C': [0.1, 1, 10], 'classifier__penalty': ['l2']},
        {'classifier__C': [0.1, 1, 10], 'classifier__penalty': ['l1'], 'classifier__solver': ['liblinear']},
        {'classifier__C': [0.1, 1, 10], 'classifier__penalty': ['elasticnet'], 'classifier__l1_ratio': [0.4, 0.5, 0.6], 'classifier__solver': ['saga']}
    ],
    'random_forest': [
        {'classifier__n_estimators': [50, 100, 200]}
    ],
    'decision_tree': [
        {'classifier__max_depth': [None, 5, 10]}
    ]
}





In [71]:
x_train_pp=ct.fit_transform(x_train)

In [72]:
# Grid search for each model
for models in pipelines.keys():
    print(f"Tuning hyperparameters for {models}...")
    grid_search = GridSearchCV(pipelines[models], param_grids[models], cv=5)
    grid_search.fit(x_train_pp, y_train)  
    print(f"Best parameters found for {models}: {grid_search.best_params_}")
    print(f"Best score found for {models}: {grid_search.best_score_}")

Tuning hyperparameters for logistic_regression...
Best parameters found for logistic_regression: {'classifier__C': 10, 'classifier__penalty': 'l1', 'classifier__solver': 'liblinear'}
Best score found for logistic_regression: 0.8004247725454678
Tuning hyperparameters for random_forest...
Best parameters found for random_forest: {'classifier__n_estimators': 200}
Best score found for random_forest: 0.993454632192039
Tuning hyperparameters for decision_tree...
Best parameters found for decision_tree: {'classifier__max_depth': None}
Best score found for decision_tree: 0.9762330594138608


In [73]:
#Fitting best model with  best parameters

In [74]:

model=Pipeline([("ct",ct),("algo",LogisticRegression(C = 10, penalty = 'l1', solver = 'liblinear'))])


In [75]:
model.fit(x_train,y_train)

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('nominal',
                                                  Pipeline(steps=[('imputing_c',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('Encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['BattingTeam', 'BowlingTeam'], dtype='object')),
                                                 ('numerical',
                                                  Pipeline(steps=[('imputing_n',
                                                                   SimpleImputer()),
                                                                  ('scaling',
                                                                   StandardScaler())]),
                                                  Index(['runs_left', 'balls_left', 'target', 'crr', 'rrr'], dtype='object'))])),
                ('algo',
                 LogisticRegression(C=10, penalty='l1', solver='liblinear'))])

In [76]:
y_pred = model.predict(x_test)

In [77]:

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.80      0.76      0.78      7988
           1       0.80      0.84      0.82      9200

    accuracy                           0.80     17188
   macro avg       0.80      0.80      0.80     17188
weighted avg       0.80      0.80      0.80     17188



In [78]:
model.predict_proba(x_test)[2]

array([0.07341287, 0.92658713])

In [79]:
#Dumping the model
pickle.dump(model,open(r"C:\Users\hp\Downloads\IPL project\ipl2.pkl",'wb'))